# NE204 Lab 2

# Importing libraries and defining functions

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
from scipy.optimize import curve_fit
from scipy.signal import find_peaks, peak_widths

In [ ]:
def ViewPulses(filelocation, events, XRange, preTrgrDly):
    with h5py.File(filelocation, 'r') as f:
        for a in range(events):
            pulse = np.array(f['raw_data'][a])  # type: ignore
            baseline = np.average(pulse[:preTrgrDly-100])
            pulse = pulse - baseline
            plt.plot(range(XRange[0], XRange[1]), pulse[XRange[0]:XRange[1]])
        plt.xlabel("Counts (4 nanoseconds per)")
        plt.ylabel("Units proportional to voltage")

def CreateSpectra(filelocation, events, preTrgrDly, risetime):
    with h5py.File(filelocation, 'r') as f:
        spectra = []
        for a in range(events):
            pulse = np.array(f['raw_data'][a])  # type: ignore
            baseline = np.average(pulse[:preTrgrDly-100])
            pulse = pulse - baseline
            sum = np.sum(pulse[preTrgrDly-100:preTrgrDly+risetime-100])
            if sum > 0:
                spectra.append(sum)
        return(spectra)

def Gaussian(x, a, b, c, d):
    return a*np.exp(-(x-b)**2/(2*c**2))+d

def PeakInfo(hist, d, promF):
    peaklocations, _ = find_peaks(hist, distance=d, prominence=int(np.amax(hist))/promF)
    widths = peak_widths(hist, peaklocations, rel_height=0.5)
    Xranges = []
    coefficients = []
    numberofpeaks = 0
    FWHMs = []
    for i in range(len(peaklocations)):
        try:
            left = int(peaklocations[i]-widths[0][i])
            right = int(peaklocations[i]+widths[0][i])
            x = np.linspace(left, right, right-left)
            popt, pcov = curve_fit(Gaussian, x, hist[left:right], [hist[peaklocations[i]], peaklocations[i], widths[0][i], 0])
            popt[2] = np.abs(popt[2])
            Xranges.append(x)
            coefficients.append(popt)
            numberofpeaks += 1
            FWHMs.append(widths[0][i])
        except:
            pass
    return Xranges, coefficients, numberofpeaks, FWHMs


# Plotting raw raveforms

Specifying that rise time starts 100 before trigger (preTrgrDly-100)

In [ ]:
filelocation = r"C:\Users\Operator\Documents\Devin\Code\Data\NaI_Cs137_4cm_30.h5"
events = 50
XRange = (900,2000)
preTrgrDly = 1000
ViewPulses(filelocation, events, XRange, preTrgrDly)
plt.title('NaI: 50 pulses from Cs-137 (4cm away)')
plt.show()

In [ ]:
filelocation = r"C:\Users\Operator\Documents\Devin\Code\Data\NaI_Cs137_28cm_30.h5"
events = 50
XRange = (900,2000)
preTrgrDly = 1000
ViewPulses(filelocation, events, XRange, preTrgrDly)
plt.title('NaI: 50 pulses from Cs-137 (28 cm away)')
plt.show()

The initial pulses generally decay mostly away by about 300 counts after the trigger (1.2 microseconds), so I will use 400 as the initial guess for rise time. Don't see much of a difference in pile-up rate between 4 and 28 cm away however. Noting that since the pulse heights seem to average around 1000, integrating over 400 counts would give pulses that are almost all under 400*1000 = 400000. So I will set the histogram range to 0, 400000

In [ ]:
filelocation = r"C:\Users\Operator\Documents\Devin\Code\Data\EJ309_Cs137_60s.h5"
events = 50
XRange = (980,995)
preTrgrDly = 1000
ViewPulses(filelocation, events, XRange, preTrgrDly)
plt.title('EJ309: 50 pulses from Cs-137')
plt.show()

In [ ]:
filelocation = r"C:\Users\Operator\Documents\Devin\Code\Data\EJ309_Cf252_60s.h5"
events = 50
XRange = (980,995)
preTrgrDly = 1000
ViewPulses(filelocation, events, XRange, preTrgrDly)
plt.title('EJ309: 50 pulses from Cf-252')
plt.show()

In [ ]:
filelocation = r"C:\Users\Operator\Documents\Devin\Code\Data\EJ301_Cs137_60s.h5"
events = 50
XRange = (975,1000)
preTrgrDly = 1000
ViewPulses(filelocation, events, XRange, preTrgrDly)
plt.title('EJ301: 50 pulses from Cs-137')
plt.show()

In [ ]:
filelocation = r"C:\Users\Operator\Documents\Devin\Code\Data\CLLBC_Cs137_60s.h5"
events = 50
XRange = (900,1100)
preTrgrDly = 1000
ViewPulses(filelocation, events, XRange, preTrgrDly)
plt.title('CLLBC: 50 pulses from Cs-137')
plt.show()

Will have to return to lab to get more data with CLLBC, did not capture the whole decay with 1100 counts per event. 

# Analyzing NaI Cs-137 Data to find optimal integration period

In [ ]:
filelocation = r"C:\Users\Operator\Documents\Devin\Code\Data\NaI_Cs137_4cm_30.h5"
events = 200000
preTrgrDly = 1000
peakdistance = 10
prominencefactor = 10
risetime = 400
spectra = CreateSpectra(filelocation, events, preTrgrDly, risetime)
print(np.max(spectra))
hist, bins = np.histogram(spectra, bins = 2000, range = (0, 400000))
plt.plot(range(np.size(hist)), hist, label="risetime =" + str(risetime))
Xranges, coefficients, numberofpeaks, FWHMs = PeakInfo(hist, peakdistance, prominencefactor)
for i in range(numberofpeaks):
    plt.plot(Xranges[i], Gaussian(Xranges[i], *coefficients[i]), label="peak " + str(i+1))
    print("Centroid is at: " + str(coefficients[i][1]))
    print("FWHM is: " + str(2.35*coefficients[i][2]))
    print("% Resolution = " + str(100*2.35*coefficients[i][2]/coefficients[i][1]))
plt.legend(loc='upper right')
plt.show()

Seems like the range can be decreased to half of before. Noting that the maximum of the spectra has a value of 1,110,000. 

In [ ]:
filelocation = r"C:\Users\Operator\Documents\Devin\Code\Data\NaI_Cs137_4cm_30.h5"
risetime = 200

spectra = CreateSpectra(filelocation, events, preTrgrDly, risetime)
hist, bins = np.histogram(spectra, bins = 1000, range = (0, 200000))
plt.plot(range(np.size(hist)), hist, label="risetime =" + str(risetime))
Xranges, coefficients, numberofpeaks, FWHMs = PeakInfo(hist, peakdistance, prominencefactor)
for i in range(numberofpeaks):
    plt.plot(Xranges[i], Gaussian(Xranges[i], *coefficients[i]), label="peak " + str(i+1))
    print("Centroid is at: " + str(coefficients[i][1]))
    print("FWHM is: " + str(2.35*coefficients[i][2]))
    print("% Resolution = " + str(100*2.35*coefficients[i][2]/coefficients[i][1]))

filelocation = r"C:\Users\Operator\Documents\Devin\Code\Data\NaI_Cs137_4cm_30.h5"
risetime = 1000

spectra = CreateSpectra(filelocation, events, preTrgrDly, risetime)
hist, bins = np.histogram(spectra, bins = 1000, range = (0, 200000))
plt.plot(range(np.size(hist)), hist, label="risetime =" + str(risetime))
Xranges, coefficients, numberofpeaks, FWHMs = PeakInfo(hist, peakdistance, prominencefactor)
for i in range(numberofpeaks):
    plt.plot(Xranges[i], Gaussian(Xranges[i], *coefficients[i]), label="peak " + str(i+1))
    print("Centroid is at: " + str(coefficients[i][1]))
    print("FWHM is: " + str(2.35*coefficients[i][2]))
    print("% Resolution = " + str(100*2.35*coefficients[i][2]/coefficients[i][1]))

plt.legend(loc='upper right')
plt.show()

Comparing integration periods of 200 and 1000 counts, the spectra created from a longer rise time will have a photopeak that shows up further to the right on the graph since more of the pulse is integrated over. Setting the integration period to 1000 shows a second peak at twice the magnitude, due to integrating over two pulses. 

With 1000 bins for a range of 200000, equivalent bin size for a full range of values (up to 1110000) would require ~6000 bins.

In [ ]:
filelocation = r"C:\Users\Operator\Documents\Devin\Code\Data\NaI_Cs137_28cm_30.h5"
risetime = 200

spectra = CreateSpectra(filelocation, events, preTrgrDly, risetime)
hist, bins = np.histogram(spectra, bins = 1000, range = (0, 200000))
plt.plot(range(np.size(hist)), hist, label="risetime =" + str(risetime))
Xranges, coefficients, numberofpeaks, FWHMs = PeakInfo(hist, peakdistance, prominencefactor)
for i in range(numberofpeaks):
    plt.plot(Xranges[i], Gaussian(Xranges[i], *coefficients[i]), label="peak " + str(i+1))
    print("Centroid is at: " + str(coefficients[i][1]))
    print("FWHM is: " + str(2.35*coefficients[i][2]))
    print("% Resolution = " + str(100*2.35*coefficients[i][2]/coefficients[i][1]))

filelocation = r"C:\Users\Operator\Documents\Devin\Code\Data\NaI_Cs137_28cm_30.h5"
risetime = 1000

spectra = CreateSpectra(filelocation, events, preTrgrDly, risetime)
hist, bins = np.histogram(spectra, bins = 1000, range = (0, 200000))
plt.plot(range(np.size(hist)), hist, label="risetime =" + str(risetime))
Xranges, coefficients, numberofpeaks, FWHMs = PeakInfo(hist, peakdistance, prominencefactor)
for i in range(numberofpeaks):
    plt.plot(Xranges[i], Gaussian(Xranges[i], *coefficients[i]), label="peak " + str(i+1))
    print("Centroid is at: " + str(coefficients[i][1]))
    print("FWHM is: " + str(2.35*coefficients[i][2]))
    print("% Resolution = " + str(100*2.35*coefficients[i][2]/coefficients[i][1]))

plt.legend(loc='upper right')
plt.show()

Seems like the resolution for the Cs-137 662 keV gamma peak doesn't change drastically between having the source 4 cm away from the detector, and 28 cm away. Going to calculate the resolution for all distances we took data for, integrating over an increasing range of counts: from 200 to 1000. Going to use the full range of values for each spectra because of the different ranges over which the pulse is integrated, and using 6000 bins to get about the same range of values per bin as the two plots above.

In [ ]:
filelocations = [ 
    r"C:\Users\Operator\Documents\Devin\Code\Data\NaI_Cs137_4cm_30.h5", 
    r"C:\Users\Operator\Documents\Devin\Code\Data\NaI_Cs137_8cm_30.h5", 
    r"C:\Users\Operator\Documents\Devin\Code\Data\NaI_Cs137_12cm_30.h5",
    r"C:\Users\Operator\Documents\Devin\Code\Data\NaI_Cs137_16cm_30.h5",
    r"C:\Users\Operator\Documents\Devin\Code\Data\NaI_Cs137_20cm_30.h5",
    r"C:\Users\Operator\Documents\Devin\Code\Data\NaI_Cs137_24cm_30.h5",
    r"C:\Users\Operator\Documents\Devin\Code\Data\NaI_Cs137_28cm_30.h5",
    ]

filenames = []
data = []
for a in filelocations:
    row = []
    filelocation = a
    filenames.append(os.path.split(a)[1])
    events = 200000
    preTrgrDly = 1000
    peakdistance = 10
    prominencefactor = 10
    for b in range(9):
        resolutions = []
        risetime = 100*(b+2)
        spectra = CreateSpectra(filelocation, events, preTrgrDly, risetime)
        hist, bins = np.histogram(spectra, bins = 6000)
        Xranges, coefficients, numberofpeaks, FWHMs = PeakInfo(hist, peakdistance, prominencefactor)
        for i in range(numberofpeaks):
            resolutions.append(100*2.35*coefficients[i][2]/coefficients[i][1])
        row.append(np.average(resolutions))    
    data.append(row)

df = pd.DataFrame(data, index=filenames, columns=100*(np.array(range(9))+2))
df.style.background_gradient(axis=None)



This heatmap of the resolutions of the Cs-137 662 keV peak show that generally, optimal resolution results from integrating the pulse over 400 counts, starting from 100 counts before the trigger. One can see that rise times that are especially long (between 800 and 1000 counts) impair resolution the most for the data taken with the source 4 cm away.

For 200000 events per, this process took 49 minutes.

# Measuring energy resolutions

In [ ]:
total_peakinfo = []
energies = []

Will add to this list to create energy calibration

In [ ]:
filelocation = r"C:\Users\Operator\Documents\Devin\Code\Data\NaI_Co60_120s.h5"
risetime = 400
prominencefactor = 10
peakinfo = []

spectra = CreateSpectra(filelocation, events, preTrgrDly, risetime)
hist, bins = np.histogram(spectra, bins = 500, range = (0, 200000))
plt.plot(range(np.size(hist)), hist, label="risetime =" + str(risetime))
Xranges, coefficients, numberofpeaks, FWHMs = PeakInfo(hist, peakdistance, prominencefactor)
for i in range(numberofpeaks):
    plt.plot(Xranges[i], Gaussian(Xranges[i], *coefficients[i]), label="peak " + str(i+1))
    peakinfo.append([coefficients[i][1], 100*2.35*coefficients[i][2]/coefficients[i][1]])
    print("Centroid is at: " + str(coefficients[i][1]))
    print("FWHM is: " + str(2.35*coefficients[i][2]))
    print("% Resolution = " + str(100*2.35*coefficients[i][2]/coefficients[i][1]))
plt.legend(loc='upper right')
plt.show()

In [ ]:
print(peakinfo)
total_peakinfo.extend(peakinfo)
energies.extend([1173.2, 1332.5])

In [ ]:
filelocation = r"C:\Users\Operator\Documents\Devin\Code\Data\NaI_Eu152_120s.h5"
prominencefactor = 20
peakinfo = []

spectra = CreateSpectra(filelocation, events, preTrgrDly, risetime)
hist, bins = np.histogram(spectra, bins = 500, range = (0,200000))
plt.plot(range(np.size(hist)), hist, label="risetime =" + str(risetime))
Xranges, coefficients, numberofpeaks, FWHMs = PeakInfo(hist, peakdistance, prominencefactor)
for i in range(numberofpeaks):
    plt.plot(Xranges[i], Gaussian(Xranges[i], *coefficients[i]), label="peak " + str(i+1))
    peakinfo.append([coefficients[i][1], 100*2.35*coefficients[i][2]/coefficients[i][1]])
    print("Centroid is at: " + str(coefficients[i][1]))
    print("FWHM is: " + str(2.35*coefficients[i][2]))
    print("% Resolution = " + str(100*2.35*coefficients[i][2]/coefficients[i][1]))
plt.legend(loc='upper right')
plt.show()

Will not add peaks 1 and 2 to list

In [ ]:
print(peakinfo[2:])
total_peakinfo.extend(peakinfo[2:])
energies.extend([121.8, 244.7, 344.3, 778.9, 964.1, 1112.1, 1408])

In [ ]:
filelocation = r"C:\Users\Operator\Documents\Devin\Code\Data\NaI_Na22_120s.h5"
prominencefactor = 10
peakinfo = []

spectra = CreateSpectra(filelocation, events, preTrgrDly, risetime)
hist, bins = np.histogram(spectra, bins = 500, range = (0,200000))
plt.plot(range(np.size(hist)), hist, label="risetime =" + str(risetime))
Xranges, coefficients, numberofpeaks, FWHMs = PeakInfo(hist, peakdistance, prominencefactor)
for i in range(numberofpeaks):
    plt.plot(Xranges[i], Gaussian(Xranges[i], *coefficients[i]), label="peak " + str(i+1))
    peakinfo.append([coefficients[i][1], 100*2.35*coefficients[i][2]/coefficients[i][1]])
    print("Centroid is at: " + str(coefficients[i][1]))
    print("FWHM is: " + str(2.35*coefficients[i][2]))
    print("% Resolution = " + str(100*2.35*coefficients[i][2]/coefficients[i][1]))
plt.legend(loc='upper right')
plt.show()

Will not add peaks 1 and 2 to list (511 keV peak should not be used)

In [ ]:
print(peakinfo[2:])
total_peakinfo.extend(peakinfo[2:])
energies.extend([1274.5])

In [ ]:
filelocation = r"C:\Users\Operator\Documents\Devin\Code\Data\NaI_Ba133_120s.h5"
prominencefactor = 15
peakinfo = []

spectra = CreateSpectra(filelocation, events, preTrgrDly, risetime)
hist, bins = np.histogram(spectra, bins = 200, range = (0,80000))
plt.plot(range(np.size(hist)), hist, label="risetime =" + str(risetime))
Xranges, coefficients, numberofpeaks, FWHMs = PeakInfo(hist, peakdistance, prominencefactor)
for i in range(numberofpeaks):
    plt.plot(Xranges[i], Gaussian(Xranges[i], *coefficients[i]), label="peak " + str(i+1))
    peakinfo.append([coefficients[i][1], 100*2.35*coefficients[i][2]/coefficients[i][1]])
    print("Centroid is at: " + str(coefficients[i][1]))
    print("FWHM is: " + str(2.35*coefficients[i][2]))
    print("% Resolution = " + str(100*2.35*coefficients[i][2]/coefficients[i][1]))
plt.legend(loc='upper right')
plt.show()

I don't believe the second peak is a gamma peak

In [ ]:
del peakinfo[1]
print(peakinfo)
total_peakinfo.extend(peakinfo)
energies.extend([81, 302, 356])


In [ ]:
all_peaks = np.array(total_peakinfo)[:,0]
plt.scatter(all_peaks, energies)
def Linear(x, a, b):
    return a*x+b
popt, pcov = curve_fit(Linear, np.array(total_peakinfo)[:,0], energies)
print(popt)
x = np.linspace(int(np.min(all_peaks)), int(np.max(all_peaks)), int(np.max(all_peaks)-np.min(all_peaks)))
plt.plot(x, Linear(x, *popt))
plt.xlabel("Channels")
plt.ylabel("Energies in keV")
plt.show()


In [ ]:
all_resolutions = np.array(total_peakinfo)[:,1]
plt.scatter(all_peaks, all_resolutions)
plt.show()

In [ ]:
plt.scatter(np.log(all_peaks), np.log(all_resolutions))
plt.show()